In [3]:
from pymongo import MongoClient
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import os
import tqdm


In [4]:
def build_qa_chain():

    # Récupération des infos pour la connexion à la bdd
    mongo_uri = os.getenv("MONGO_URI")
    db_name = os.getenv("DB_NAME")
    collection_name = os.getenv("COLLECTION_NAME")
    text_field = os.getenv("TEXT_FIELD", "texte")

    # Connexion à MongoDB
    client = MongoClient(mongo_uri)
    collection = client[db_name][collection_name]
    texts = [doc[text_field] for doc in collection.find({}, {text_field: 1}) if text_field in doc]

    # Séparation des textes
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = [Document(page_content=chunk) for text in texts for chunk in splitter.split_text(text)]
    
    # Vectorisation des textes
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(docs, embeddings)
    
    retriever = vectorstore.as_retriever()
    # llm = ChatOpenAI(temperature=0)
    llm_ollama = Ollama(model="mistral")
    
    return RetrievalQA.from_chain_type(llm=llm_ollama, retriever=retriever)

In [5]:
build_qa_chain()

/tmp/ipykernel_40491/116804841.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/abdoul/rag_lawyer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_40491/116804841.py:24: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama pac

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=Ollama(model='mistral'), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fbfddb158b0>, search_kwargs={}))

In [ ]:
# Récupération des infos pour la connexion à la bdd
mongo_uri = os.getenv("MONGO_URI")
db_name = os.getenv("DB_NAME")
collection_name = os.getenv("COLLECTION_NAME")
text_field = os.getenv("TEXT_FIELD", "texte")

# Connexion à MongoDB
client = MongoClient(mongo_uri)
collection = client[db_name][collection_name]
texts = [doc[text_field] for doc in collection.find({}, {text_field: 1}) if text_field in doc]    

In [19]:
# Séparation des textes
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Forcer l'encodage en UTF8
def nettoyer_utf8(texte):
    return texte.encode("utf-8", errors="ignore").decode("utf-8")

docs = [
    Document(page_content=nettoyer_utf8(chunk))
    for text in texts
    for chunk in splitter.split_text(text)
]

In [24]:
docs

[Document(metadata={}, page_content="La protection des personnes physiques à l'égard du traitement des données à caractère personnel est un droit fondamental. L'article 8, paragraphe 1, de la Charte des droits fondamentaux de l'Union européenne (ci-après dénommée «Charte») et l'article 16, paragraphe 1, du traité sur le fonctionnement de l'Union européenne disposent que toute personne a droit à la protection des données à caractère personnel la concernant.Les principes et les règles régissant la protection des personnes physiques à"),
 Document(metadata={}, page_content="régissant la protection des personnes physiques à l'égard du traitement des données à caractère personnel les concernant devraient, quelle que soit la nationalité ou la résidence de ces personnes physiques, respecter leurs libertés et droits fondamentaux, en particulier leur droit à la protection des données à caractère personnel. Le présent règlement vise à contribuer à la réalisation d'un espace de liberté, de sécuri

In [35]:
doc = [Document(metadata={},page_content="test")]

In [45]:
# Vectorisation des textes
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(doc, embeddings)

In [49]:
# Affiche les documents stockés dans FAISS
print(vectorstore.docstore._dict)  # ou list(vectorstore.docstore._dict.values())

{'65a921fd-67ec-408a-a0f7-e30ba8cef536': Document(id='65a921fd-67ec-408a-a0f7-e30ba8cef536', metadata={}, page_content='test')}


In [47]:
retriever = vectorstore.as_retriever()
# llm = ChatOpenAI(temperature=0)
llm_ollama = Ollama(model="mistral")

/tmp/ipykernel_27561/2374047528.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm_ollama = Ollama(model="mistral")
